In [1]:
import kaolin
import torch
import math
import matplotlib
import os
import numpy as np
import matplotlib.pyplot as plt
import polyscope as ps
import trimesh

from scipy.spatial import Voronoi, voronoi_plot_2d, Delaunay
from io import BytesIO
from PIL import Image
import sdfpred_utils.sdfpred_utils as su
import sdfpred_utils.sdf_MLP as mlp
import sdfpred_utils.sdf_functions as sdf
import sdfpred_utils.loss_functions as lf
import sdfpred_utils.Steik_data3d as sd3d
import sdfpred_utils.Steik_Loss as sl
import sdfpred_utils.Steik_utils as Stu 

#cuda devices
device = torch.device("cuda:0")
print("Using device: ", torch.cuda.get_device_name(device))

#default tensor types
torch.set_default_tensor_type(torch.cuda.DoubleTensor)

multires = 2
input_dims = 3
lr_sites = 0.005/2
lr_model = 0.00005*2
destination = "./images/autograd/3Dsteik/"


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using device:  NVIDIA GeForce RTX 3090


In [2]:
#currently sites are between -5 and 5 in all 3 dimensions
# check if sites exists
#num_centroids = 16*16*16
#num_centroids =16*16*16*8
num_centroids = 16*16*16#*8 
#num_centroids = 128*128*128

site_fp = f'sites_{num_centroids}_{input_dims}.pt'

# if os.path.exists(site_fp):
#     sites = torch.load(site_fp)
#     print("Sites loaded:", sites.shape)
# else:
print("Creating new sites")
noise_scale = 0.1
domain_limit = 1.5
x = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids**(1/3))))
y = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids**(1/3))))
z = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids**(1/3))))
meshgrid = torch.meshgrid(x, y, z)
meshgrid = torch.stack(meshgrid, dim=3).view(-1, 3)
print("Meshgrid shape:", meshgrid.shape)
print("Meshgrid 1st 5:", meshgrid[:5])
#add noise to meshgrid
meshgrid += torch.randn_like(meshgrid) * noise_scale
print("Meshgrid 1st 5:", meshgrid[:5])
sites = meshgrid.to(device, dtype=torch.double).requires_grad_(True)

#print min max sites 
print("Sites min:", sites.min(dim=0).values)
print("Sites max:", sites.max(dim=0).values)
print("Sites shape:", sites.shape)

#sites = su.createCVTgrid(num_centroids=num_centroids, dimensionality=input_dims)
#save the initial sites torch tensor
#torch.save(sites, site_fp)


def plot_voronoi_3d(sites, xlim=5, ylim=5, zlim=5):
    import numpy as np
    import pyvoro
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    # initialize random number generator
    rng = np.random.default_rng(11)
    # create a set of points in 3D
    points = sites.detach().cpu().numpy()

    # use pyvoro to compute the Voronoi tessellation
    # the second argument gives the the axis limits in x,y and z direction
    # in this case all between 0 and 1.
    # the third argument gives "dispersion = max distance between two points
    # that might be adjacent" (not sure how exactly this works)
    voronoi = pyvoro.compute_voronoi(points,[[-xlim,xlim],[-ylim,ylim],[-zlim,zlim]],1)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # for each Voronoi cell, plot all the faces of the corresponding polygon
    for vnoicell in voronoi:
        faces = []
        # the vertices are the corner points of the Voronoi cell
        vertices = np.array(vnoicell['vertices'])
        # cycle through all faces of the polygon
        for face in vnoicell['faces']:
            faces.append(vertices[np.array(face['vertices'])])
            
        # join the faces into a 3D polygon
        polygon = Poly3DCollection(faces, alpha=0.5, 
                                facecolors=rng.uniform(0,1,3),
                                linewidths=0.5,edgecolors='black')
        ax.add_collection3d(polygon)
    
    ax.set_xlim([-xlim,xlim])
    ax.set_ylim([-ylim,ylim])
    ax.set_zlim([-zlim,zlim])
            
    plt.show()
    

def polyscope_sdf(model,i):
    # Render the SDF as an implicit surface (zero-level set)
    def model_sdf(pts):
        pts_tensor = torch.tensor(pts, dtype=torch.float64, device=device)
        sdf_values = model(pts_tensor)
        sdf_values_np = sdf_values.detach().cpu().numpy().flatten()  # Convert to NumPy
        
        return sdf_values_np

    ps.render_implicit_surface(f"SDF Surface {i}", model_sdf, mode="sphere_march", enabled=True, subsample_factor=2)
    
def polyscope_marching_tet(model, sites, i=""):
    import scipy.spatial as spatial
    from scipy.spatial import Delaunay

    sites_np = sites.detach().cpu().numpy()
    tri = Delaunay(sites_np)
    delaunay_vertices =torch.tensor(np.array(tri.simplices), device=device)
    sdf_values = model(sites)


    # Assuming sites is a PyTorch tensor of shape [M, 3]
    sites = sites.unsqueeze(0)  # Now shape [1, M, 3]

    # Assuming SDF_Values is a PyTorch tensor of shape [M]
    sdf_values = sdf_values.unsqueeze(0)  # Now shape [1, M]

    marching_tetrehedra_mesh = kaolin.ops.conversions.marching_tetrahedra(sites, delaunay_vertices, sdf_values, return_tet_idx=False)
    print(marching_tetrehedra_mesh)
    vertices_list, faces_list = marching_tetrehedra_mesh
    vertices = vertices_list[0]
    faces = faces_list[0]
    vertices_np = vertices.detach().cpu().numpy()  # Shape [N, 3]
    faces_np = faces.detach().cpu().numpy()  # Shape [M, 3] (triangles)
    ps.register_surface_mesh(f"Marching Tetrahedra Mesh{i}", vertices_np, faces_np)





#plot_voronoi_3d(sites)

Creating new sites
Meshgrid shape: torch.Size([4096, 3])
Meshgrid 1st 5: tensor([[-1.5000, -1.5000, -1.5000],
        [-1.5000, -1.5000, -1.3000],
        [-1.5000, -1.5000, -1.1000],
        [-1.5000, -1.5000, -0.9000],
        [-1.5000, -1.5000, -0.7000]])
Meshgrid 1st 5: tensor([[-1.3439, -1.6130, -1.3673],
        [-1.4276, -1.5825, -1.3349],
        [-1.4299, -1.5721, -1.2085],
        [-1.4404, -1.5605, -0.7878],
        [-1.4415, -1.5551, -0.8328]])
Sites min: tensor([-1.7681, -1.7501, -1.7641], grad_fn=<MinBackward0>)
Sites max: tensor([1.8419, 1.7887, 1.8176], grad_fn=<MaxBackward0>)
Sites shape: torch.Size([4096, 3])


/home/wylliam/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1678402412426/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
ps.init()
#ps_cloud = ps.register_point_cloud("initial_cvt_grid",sites.detach().cpu().numpy())

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 3.3.0 NVIDIA 570.133.07


In [4]:
# # Load the mesh
mesh = ["bunny", "Resources/stanford-bunny.obj"]
# # #mesh = ["staryu", "Resources/staryu.obj"]
# mesh = ["chair", "Resources/chair_low.obj"]

# bunny = trimesh.load(mesh[1])
# #target_points = bunny.sample(16*16*16)

# target_points = bunny.sample(16*16*16 * 4)
# target_points = target_points - np.mean(target_points, axis=0)
# target_points = target_points / np.max(np.abs(target_points))


# target_points = torch.tensor(target_points, device=device)
# print("Target points:", target_points.shape)
# min_target = target_points.min(0)[0]
# max_target = target_points.max(0)[0]
# print("min_target", min_target)
# print("max_target", max_target)

# ps.register_point_cloud("Target_points",target_points.detach().cpu().numpy())

# #ps.show()


In [5]:
shape_type = 'bunny'
res = 128 # has to be even
example_idx = 0
sample_type = 'grid'
n_samples = 1
n_points = 16*16*16*8
#TODO: change to 3D look into Steik to make it work 
# dataset = sd2d.get2D_dataset(n_points, n_samples, res, sample_type, 0.005, shape_type=shape_type)  # BasicShape2D(100, 20, res=50)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False, num_workers=0, pin_memory=False)

dataset = sd3d.ReconDataset("Resources/stanford-bunny.obj", n_points, n_samples=n_samples)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0 ,pin_memory=False)
data = next(iter(dataloader))
# print("Data keys: ", data.keys())
# mnfld_points, normals_gt, nonmnfld_dist_gt, nonmnfld_points, nonmnfld_n_gt= data['points'].to(device), data['mnfld_n'].to(device), \
#                                                                             data['nonmnfld_dist'].to(device), \
#                                                                             data['nonmnfld_points'].to(device), data['nonmnfld_n'].to(device),
mnfld_points, mnfld_n_gt, nonmnfld_points = data['points'].to(device), data['mnfld_n'].to(device),  data['nonmnfld_points'].to(device)

mnfld_points = mnfld_points[0]
mnfld_points = mnfld_points.double()
mnfld_n_gt = mnfld_n_gt[0]
mnfld_n_gt = mnfld_n_gt.double()
nonmnfld_points = nonmnfld_points[0]
nonmnfld_points = nonmnfld_points.double()

#print min max values of manifold

print("Manifold points shape: ", mnfld_points.shape)
print("Manifold points normals GT shape: ", mnfld_n_gt.shape)
print("Non-manifold points shape: ", nonmnfld_points.shape)
print("Manifold points min:", torch.min(mnfld_points, dim=0).values)
print("Manifold points max:", torch.max(mnfld_points, dim=0).values)
mnfld_points.requires_grad_()
nonmnfld_points.requires_grad_()

small_mnfl_points = mnfld_points*0.99
big_mnfl_points = mnfld_points*1.01

ps.register_point_cloud("mnfld", mnfld_points.detach().cpu().numpy())
#ps.register_point_cloud("normals", mnfld_n_gt.detach().cpu().numpy())
#ps.register_point_cloud("big_mnfl_points", big_mnfl_points.detach().cpu().numpy())
#ps.register_point_cloud("small_mnfl_points", small_mnfl_points.detach().cpu().numpy())
#ps.register_point_cloud("non mnfld", nonmnfld_points.detach().cpu().numpy())
ps.show()


Manifold points shape:  torch.Size([32768, 3])
Manifold points normals GT shape:  torch.Size([32768, 3])
Non-manifold points shape:  torch.Size([32768, 3])
Manifold points min: tensor([-0.7309, -0.6531, -0.7540])
Manifold points max: tensor([0.9407, 1.0000, 0.5411])


In [6]:
# import open3d as o3d
# import numpy as np

# # Load point cloud
# mesh = o3d.io.read_triangle_mesh("Resources/stanford-bunny.obj")
# pcd = mesh.sample_points_uniformly(16*16*16*8)

# points = np.asarray(pcd.points, dtype=np.float32)
# #min max values of points
# print("Points min:", points.min(axis=0))
# print("Points max:", points.max(axis=0))

# # center and scale point cloud
# center = points.mean(axis=0)
# points = points - center[None, :]
# # self.scale = np.linalg.norm(points, axis=-1).max(-1)
# scale = np.abs(points).max()
# points = points / scale
# pcd.points = o3d.utility.Vector3dVector(points)


# #min max values of points
# print("Points min:", points.min(axis=0))
# print("Points max:", points.max(axis=0))


# # Estimate normals
# pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))
# # Flip normals towards a common viewpoint (optional)
# pcd.orient_normals_consistent_tangent_plane(k=30)
# # Visualize
# #o3d.visualization.draw_geometries([pcd], point_show_normal=True)


# # Convert point cloud to numpy
# points = np.asarray(pcd.points)
# normals = np.asarray(pcd.normals)

# # Define offset distance
# d = 0.003   # Adjust as needed

# # Create outer and inner point clouds
# points_outer = points + d * normals
# points_inner = points - d * normals

# # Create Open3D point clouds
# pcd_outer = o3d.geometry.PointCloud()
# pcd_outer.points = o3d.utility.Vector3dVector(points_outer)

# pcd_inner = o3d.geometry.PointCloud()
# pcd_inner.points = o3d.utility.Vector3dVector(points_inner)

# ps.register_point_cloud("points",points)
# ps.register_point_cloud("points_inner", np.asarray(pcd_inner.points))
# ps.register_point_cloud("points_outer", np.asarray(pcd_outer.points))



# # uniformely sample from points
# num_points = points.shape[0]
# indices = torch.randperm(num_points)[:16**3].cpu().numpy()
# # Subsample
# sub_points = points[indices]  # (N^3, 3)
# sub_points_inner = points_inner[indices]  # (N^3, 3)
# sub_points_outer = points_outer[indices]  # (N^3, 3)

# ps.register_point_cloud("sub_points",sub_points)
# ps.register_point_cloud("sub_points_inner", sub_points_inner)
# ps.register_point_cloud("sub_points_outer", sub_points_outer)





# ps.show()

# # Save or visualize
# o3d.io.write_point_cloud("bunny_outer.ply", pcd_outer)
# o3d.io.write_point_cloud("bunny_inner.ply", pcd_inner)
# o3d.visualization.draw_geometries([pcd_inner, pcd_outer])



In [7]:
model = mlp.Decoder(multires=multires, input_dims=input_dims).to(device)
radius = 1
it = 1000
#model_path = 'models_resources/pretrained_sphere_small.pth'
model_path = f'models_resources/pretrained_sphere_{radius}.pth'
#model_path = f'models_resources/pc_bunny_{it}.pth'
#model_path = 'models_resources/pc_chair.pth'



if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
    print('loaded model')   
else:
    print("no model found, pretraining")
    model.pre_train_sphere(it*3, radius)
    
    #model.pre_train_pc_scaling(it*10, mnfld_points)
    #model.pre_train_pc_scaling(it*3, mnfld_points, lr = 0.00001)
    
    polyscope_marching_tet(model, sites, i=f"{it}_scaling")
    
    #model.pre_train_pc(it*6)
    #polyscope_marching_tet(model, sites, i=f"{it}_pcn")
    
    torch.save(model.state_dict(),model_path)

#polyscope_marching_tet(model, sites, i=f"_{it}_pc")

#ps.show()


#polyscope_marching_tet(model, sites)
#final_mesh = su.get_zero_crossing_mesh_3d(sites, model)
#ps.register_surface_mesh("Zero-Crossing faces pretrained", final_mesh[0], final_mesh[1])


loaded model


In [8]:
def upsampling_vectorized(sites, tri=None, vor=None, model=None):
    sdf_values = model(sites)
    sites_np = sites.detach().cpu().numpy()

    if vor is None:
        vor = Voronoi(sites_np)
    if tri is None:
        tri = Delaunay(sites_np)
    
    neighbors = torch.tensor(np.array(vor.ridge_points), device=device)
    
    # Extract the SDF values for each site in the pair
    sdf_i = sdf_values[neighbors[:, 0]]  # First site in each pair
    sdf_j = sdf_values[neighbors[:, 1]]  # Second site in each pair
    # Find the indices where SDF values have opposing signs or one is zero
    mask_zero_crossing_sites = (sdf_i * sdf_j <= 0).squeeze()
    sites_to_upsample = torch.unique(neighbors[mask_zero_crossing_sites].view(-1))
    
    print("Sites to upsample ",sites_to_upsample.shape)
    
    tet_centroids = sites[sites_to_upsample]

    # Tetrahedron relative positions (unit tetrahedron)
    basic_tet_1 = torch.tensor([[1, 1, 1]], device=device, dtype=torch.float64)
    basic_tet_1 = basic_tet_1.repeat(len(tet_centroids), 1)
    basic_tet_2 = torch.tensor([-1, -1, 1], device=device, dtype=torch.float64)    
    basic_tet_2 = basic_tet_2.repeat(len(tet_centroids), 1)
    basic_tet_3 = torch.tensor([-1, 1, -1], device=device, dtype=torch.float64)    
    basic_tet_3 = basic_tet_3.repeat(len(tet_centroids), 1)
    basic_tet_4 = torch.tensor([1, -1, -1], device=device, dtype=torch.float64)
    basic_tet_4 = basic_tet_4.repeat(len(tet_centroids), 1)


    #compute scale based on cell volume
    #centroids = torch.tensor(np.array([vor.vertices[vor.regions[vor.point_region[i]]].mean(axis=0) for i in range(len(sites_np))]), device=device)
    centroids = lf.compute_voronoi_cell_centers_index_based_torch(tri)
    
    #centroids = torch.tensor(np.array(centroids), device=sites.device, dtype=sites.dtype)
    cells_vertices = [vor.vertices[vor.regions[vor.point_region[i]]] for i in range(len(sites_np))]

    #compute the distance between each centroid  and each vertex in cells_vertices row
    distances = []
    for i in range(len(cells_vertices)):
        min_dist = 100000000000
        for j in range(len(cells_vertices[i])):
            dist = torch.norm(centroids[i] - torch.tensor(cells_vertices[i][j], device=device), p=2)
            if dist < min_dist:
                min_dist = dist
        distances.append(min_dist)
    distances = torch.tensor(distances, device=device)
 
    
    scale = distances[sites_to_upsample] / 2
    
    scale = scale.unsqueeze(1)
    
    
    new_sites = torch.cat((tet_centroids + basic_tet_1 * scale, tet_centroids + basic_tet_2 * scale, tet_centroids + basic_tet_3 * scale, tet_centroids + basic_tet_4 * scale), dim=0)

    updated_sites = torch.cat((sites, new_sites), dim=0)

    return updated_sites
                



In [9]:
cvt_loss_values = []
min_distance_loss_values = []
chamfer_distance_loss_values = []
eikonal_loss_values = []
domain_restriction_loss_values = []
sdf_loss_values = []
div_loss_values = []
loss_values = []


def autograd(sites, model, max_iter=100, stop_train_threshold=1e-6, upsampling=0, lambda_weights = [0.1,1.0,0.1,0.1,1.0,1.0,0.1]):
    optimizer = torch.optim.Adam([
    {'params': [p for _, p in model.named_parameters()], 'lr': 5e-5},
    {'params': [sites], 'lr': lr_sites}
], betas=(0.9, 0.999))

    prev_loss = float("inf")
    best_loss = float("inf")
    upsampled = 0.0
    epoch = 0
    lambda_cvt = lambda_weights[0]
    # lambda_pc = lambda_weights[1]
    lambda_min_distance = lambda_weights[2]
    # lambda_laplace = lambda_weights[3]
    lamda_chamfer = lambda_weights[4]
    lambda_eikonal = lambda_weights[5]
    lambda_domain_restriction = lambda_weights[6]
    # lambda_target_points = lambda_weights[7]
    lambda_sdf = 5e3
    lambda_div = 1e3
    lambda_eikonal = 5e1
    best_sites = sites.clone()
    best_sites.best_loss = best_loss
    
    while epoch <= max_iter:
        optimizer.zero_grad()
        
        # compute voronoi and delaunay once for each epoch and pass it around
        # Compute Voronoi diagram
        sites_np = sites.detach().cpu().numpy()
        #vor = Voronoi(sites_np)
        tri = Delaunay(sites_np)
                
        vertices_to_compute, bisectors_to_compute = su.compute_zero_crossing_vertices_3d(sites, None, tri, model)
        #vertices_to_compute, bisectors_to_compute = su.compute_zero_crossing_vertices_3d(sites, vor, tri, model)
        
        vertices = su.compute_vertices_3d_vectorized(sites, vertices_to_compute)    
        bisectors = su.compute_all_bisectors_vectorized(sites, bisectors_to_compute)
        points = torch.cat((vertices, bisectors), 0)

        # Compute losses       
        #cvt_loss = lf.compute_cvt_loss_vectorized_voronoi(sites, vor)
        cvt_loss = lf.compute_cvt_loss_vectorized_delaunay(sites, tri)
        
        #min_distance_loss = lf.sdf_weighted_min_distance_loss(model, sites)
        print("mnfld", mnfld_points.shape)
        print("points", points.shape) 
        
        from pytorch3d.loss import chamfer_distance
        chamfer_loss, _ = chamfer_distance(mnfld_points.unsqueeze(0).detach(), points.unsqueeze(0))
        print(f"After Chamfer loss PYTORCH3D {chamfer_loss} : Allocated: {torch.cuda.memory_allocated() / 1e6} MB,\
                  Reserved: {torch.cuda.memory_reserved() / 1e6} MB")
            
        
        # chamfer_loss = lf.chamfer_distance(mnfld_points, points)
        
        sites_loss = (
            lambda_cvt * cvt_loss +
            lamda_chamfer * chamfer_loss
        )
        print(f"After site loss and Before model loss : Allocated: {torch.cuda.memory_allocated() / 1e6} MB,\
                  Reserved: {torch.cuda.memory_reserved() / 1e6} MB")
            
        # Compute model loss               
        non_manifold_pred = model(nonmnfld_points)
        manifold_pred = model(mnfld_points)
        div_loss = torch.tensor([0.0], device=mnfld_points.device)
        # compute gradients for div (divergence), curl and curv (curvature)
        if manifold_pred is not None:
            mnfld_grad = Stu.gradient(mnfld_points, manifold_pred)
        else:
            mnfld_grad = None

        nonmnfld_grad = Stu.gradient(nonmnfld_points, non_manifold_pred)
        div_loss = torch.abs(lf.directional_div(nonmnfld_points, nonmnfld_grad)).mean() #+ mnfld_divergence_term.mean()
        eikonal_term = lf.eikonal_loss(nonmnfld_grad, mnfld_grad=mnfld_grad, eikonal_type='abs')
        sdf_term = torch.abs(manifold_pred).mean()
        domain_restriction_loss = lf.domain_restriction_sphere(mnfld_points, model, input_dim=input_dims)
        

        model_loss = (
            lambda_sdf*sdf_term +
            lambda_eikonal*eikonal_term +
            lambda_div*div_loss +
            lambda_domain_restriction * domain_restriction_loss
        )
        #print weights
        print(f"lambda_sdf: {lambda_sdf}, lambda_eikonal: {lambda_eikonal}, lambda_div: {lambda_div}, lambda_domain_restriction: {lambda_domain_restriction}")
        print(f"sdf_term: {sdf_term}, eikonal_term: {eikonal_term}, div_loss: {div_loss}, domain_restriction_loss: {domain_restriction_loss}")
        print(f"Epoch {epoch}: model_loss = {model_loss.item()}")
        
        #DIVDECAY='linear' # 'linear' | 'quintic' | 'step'
        div_decay_params = [1e2, 0.2, 1e2, 0.4, 0.0, 0.0]
        
        div_decay_params = [5e2, 0.2, 5e2, 0.8, 0.0, 0.0]
        
        # div_decay_params = [lambda_div, 0.5, lambda_div/2, 0.8, 0.0, 0.0]
        
        #div_decay_params = [300, 0.5, 100, 0.8, 0.0, 0.0]
        lambda_div = lf.update_div_weight(epoch, max_iter, lambda_div, 'linear', div_decay_params)
               

        #print weights
        print(f"cvt_loss: {cvt_loss}, chamfer_loss: {chamfer_loss}")
              #, min_distance_loss: {min_distance_loss}, 
        print(f"Epoch {epoch}: site_loss = {sites_loss.item()}")

        print(f"lambda_cvt: {lambda_cvt}, lambda_min_distance: {lambda_min_distance}, lambda_chamfer: {lamda_chamfer}")
         
        loss = model_loss + sites_loss
        loss_values.append(loss.item())
        print(f"Epoch {epoch}: loss = {loss.item()}")
        print(f"before loss.backward(): Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")

        loss.backward()
        print(f"After loss.backward(): Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")
        print("-----------------")
        
        optimizer.step()
        
        if loss.item() < best_loss:
            best_loss = loss.item()
            best_epoch = epoch
            best_sites = sites.clone()
            best_sites.best_loss = best_loss
            #if upsampled > 0:
                #print(f"UPSAMPLED {upsampled} Best Epoch {best_epoch}: Best loss = {best_loss}")
                #return best_sites
        
        if abs(prev_loss - loss.item()) < stop_train_threshold:
            print(f"Converged at epoch {epoch} with loss {loss.item()}")
            #break
        
        prev_loss = loss.item() 
        
        # if epoch>100 and (epoch // 100) == upsampled+1 and loss.item() < 0.5 and upsampled < upsampling:
        if epoch/max_iter > (upsampled+1)/(upsampling+1) and upsampled < upsampling:
            print("sites length BEFORE UPSAMPLING: ",len(sites))
            
            #new_sites = su.upsampling_inside(best_sites, model)
            #new_sites = su.adaptive_density_upsampling(best_sites, model)
            
            #sites = su.add_upsampled_sites(best_sites, new_sites)
            
            sites = upsampling_vectorized(sites, tri, None, model)
            
            sites = sites.detach().requires_grad_(True)
            #print("upsampled sites length: ",len(sites))
            
            #best_sites = sites.clone()
            #best_sites.best_loss = best_loss
            
            optimizer = torch.optim.Adam([{'params': [p for _, p in model.named_parameters()], 'lr': lr_model},
                                          {'params': [sites], 'lr': lr_sites}])
            upsampled += 1.0
            print("sites length AFTER: ",len(sites))
            
          
        if epoch % (max_iter/50) == 0:
            #print(f"Epoch {epoch}: loss = {loss.item()}")
            #print(f"Best Epoch {best_epoch}: Best loss = {best_loss}")
            #save model and sites
            site_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_sites_{num_centroids}_chamfer{lamda_chamfer}.pth'
            model_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_model_{num_centroids}_chamfer{lamda_chamfer}.pth'
            torch.save(model.state_dict(), model_file_path)
            torch.save(sites, site_file_path)
            
        epoch += 1           
        
    return best_sites

In [10]:
lambda_weights = [252,0,0,0,10.211,0,100,0]

lambda_cvt = lambda_weights[0]
lambda_sdf = lambda_weights[1]
lambda_min_distance = lambda_weights[2]
lambda_laplace = lambda_weights[3]
lamda_chamfer = lambda_weights[4]
lambda_eikonal = lambda_weights[5]
lambda_domain_restriction = lambda_weights[6]
lambda_true_points = lambda_weights[7]

max_iter = 10

site_file_path = f'{destination}{max_iter}_cvt_{lambda_cvt}_chamfer_{lamda_chamfer}_eikonal_{lambda_eikonal}.npy'
#check if optimized sites file exists
if os.path.exists(site_file_path):
    #import sites
    print("Importing sites")
    sites = np.load(site_file_path)
    sites = torch.from_numpy(sites).to(device).requires_grad_(True)
else:
    # import cProfile, pstats
    # import time
    # profiler = cProfile.Profile()
    # profiler.enable()
    with torch.profiler.profile(activities=[
            torch.profiler.ProfilerActivity.CPU,
            torch.profiler.ProfilerActivity.CUDA,
        ],
        record_shapes=False,
        with_stack=True  # Captures function calls
    ) as prof:
        sites = autograd(sites, model, max_iter=max_iter, upsampling=1, lambda_weights=lambda_weights)
        torch.cuda.synchronize()

    print(prof.key_averages().table(sort_by="self_cuda_time_total"))
    prof.export_chrome_trace("trace.json")

    
    # profiler.disable()
    # stats = pstats.Stats(profiler).sort_stats('cumtime')
    # stats.print_stats()
    # stats.dump_stats(f'{destination}{mesh[0]}{max_iter}_3d_profile_{num_centroids}_chamfer{lamda_chamfer}.prof')
    
    
    sites_np = sites.detach().cpu().numpy()
    np.save(site_file_path, sites_np)
    
    

print("Sites length: ", len(sites))
print("min sites: ", torch.min(sites))
print("max sites: ", torch.max(sites))
#ps_cloud = ps.register_point_cloud("best_optimized_cvt_grid",sites.detach().cpu().numpy())
    
lim=torch.abs(torch.max(sites)).detach().cpu().numpy()*1.1
#plot_voronoi_3d(sites,lim,lim,lim)

STAGE:2025-04-03 17:13:52 92541:92541 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


mnfld torch.Size([32768, 3])
points torch.Size([3185, 3])
After Chamfer loss PYTORCH3D 0.11348692513868075 : Allocated: 17.416704 MB,                  Reserved: 56.623104 MB
After site loss and Before model loss : Allocated: 17.417216 MB,                  Reserved: 56.623104 MB
lambda_sdf: 5000.0, lambda_eikonal: 50.0, lambda_div: 1000.0, lambda_domain_restriction: 100
sdf_term: 0.16743862734010934, eikonal_term: 0.30457573497037943, div_loss: 0.43503558288858546, domain_restriction_loss: 0.0
Epoch 0: model_loss = 1287.457506337651
cvt_loss: 0.12799401905054558, chamfer_loss: 0.11348692513868075
Epoch 0: site_loss = 33.413307793328556
lambda_cvt: 252, lambda_min_distance: 0, lambda_chamfer: 10.211
Epoch 0: loss = 1320.8708141309796
before loss.backward(): Allocated: 934.442496 MB, Reserved: 1098.907648 MB
After loss.backward(): Allocated: 60.956672 MB, Reserved: 1107.296256 MB
-----------------
mnfld torch.Size([32768, 3])
points torch.Size([2965, 3])
After Chamfer loss PYTORCH3D 0.090

STAGE:2025-04-03 17:14:05 92541:92541 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2025-04-03 17:14:05 92541:92541 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                               aten::mm         0.30%      28.603ms         1.15%     109.429ms      88.392us        1.632s        43.91%        1.671s       1.349ms          1238  
void cutlass::Kernel<cutlass_80_tensorop_d884gemm_64...         0.00%       0.000us         0.00%       0.000us       0.000us     549.633ms        14.79%     549.633ms       2.082ms           264  
void cutl

In [11]:
epoch = 80
model_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_model_{num_centroids}_chamfer{lamda_chamfer}.pth'
site_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_sites_{num_centroids}_chamfer{lamda_chamfer}.pth'
 
sites = torch.load(site_file_path)
sites_np = sites.detach().cpu().numpy()
model.load_state_dict(torch.load(model_file_path))
#
#polyscope_sdf(model)
#
print("model", model_file_path)
print("sites", site_file_path)
ps_cloud = ps.register_point_cloud(f"{epoch} epoch_cvt_grid",sites_np)



FileNotFoundError: [Errno 2] No such file or directory: './images/autograd/3Dsteik/bunny10_80_3d_sites_4096_chamfer10.211.pth'

In [ ]:
final_mesh = su.get_zero_crossing_mesh_3d(sites, model)
ps.register_point_cloud("mnfld", mnfld_points.detach().cpu().numpy())

ps.register_surface_mesh("Zero-Crossing faces final", final_mesh[0], final_mesh[1])
#polyscope_sdf(model,2)

import scipy.spatial as spatial
from scipy.spatial import Delaunay

tri = Delaunay(sites_np)
delaunay_vertices =torch.tensor(np.array(tri.simplices), device=device)
sdf_values = model(sites)


# Assuming sites is a PyTorch tensor of shape [M, 3]
sites = sites.unsqueeze(0)  # Now shape [1, M, 3]

# Assuming SDF_Values is a PyTorch tensor of shape [M]
sdf_values = sdf_values.unsqueeze(0)  # Now shape [1, M]

marching_tetrehedra_mesh = kaolin.ops.conversions.marching_tetrahedra(sites, delaunay_vertices, sdf_values, return_tet_idx=False)
print(marching_tetrehedra_mesh)
vertices_list, faces_list = marching_tetrehedra_mesh
vertices = vertices_list[0]
faces = faces_list[0]
vertices_np = vertices.detach().cpu().numpy()  # Shape [N, 3]
faces_np = faces.detach().cpu().numpy()  # Shape [M, 3] (triangles)
ps.register_surface_mesh("Marching Tetrahedra Mesh final", vertices_np, faces_np)


ps.show()


[(tensor([[-0.7514,  0.3576,  0.1730],
        [-0.7510,  0.3492,  0.1581],
        [-0.7354,  0.2278,  0.1657],
        ...,
        [ 0.0471, -0.6209,  0.1085],
        [ 0.5156, -0.6165, -0.2341],
        [ 0.5821, -0.5595, -0.2387]], grad_fn=<SumBackward1>),), (tensor([[34328,    72,   415],
        [  416,   414,   415],
        [34379, 24851, 34385],
        ...,
        [31334, 28718, 27681],
        [27684, 28721, 27681],
        [27684, 46654, 28721]]),)]


In [ ]:
def export_visualisation_3d():
    import imageio
    img_buffer_mesh = []
    img_buffer_model = []
    for i in range(int(max_iter/10)+1):
        epoch = i*int(max_iter/10)
        
        site_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_sites_{num_centroids}_chamfer{lamda_chamfer}.pth'
        model_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_model_{num_centroids}_chamfer{lamda_chamfer}.pth'
        if os.path.exists(site_file_path) and os.path.exists(model_file_path):
            print("importing sites and model")
        else:
            print("files not found")
            continue
        print("mesh of epoch: ", epoch)
        
        model.load_state_dict(torch.load(model_file_path))
    
        current_mesh = su.get_zero_crossing_mesh_3d(torch.load(site_file_path), model)
        ps.remove_all_structures()
        ps.register_surface_mesh("Zero-Crossing faces", current_mesh[0], current_mesh[1])
        ps.register_point_cloud("Mesh vertices", current_mesh[0])
        img_buffer_mesh.append(ps.screenshot_to_buffer(transparent_bg=False))
        
        ps.remove_all_structures()
        #polyscope_sdf(model)
        img_buffer_model.append(ps.screenshot_to_buffer(transparent_bg=False))


    imageio.mimsave(f'{destination}{max_iter}_3d_{num_centroids}_optimization_mesh.gif',img_buffer_mesh, fps=1, duration=1, loop=0)
    imageio.mimsave(f'{destination}{max_iter}_3d_{num_centroids}_optimization_sdf.gif', img_buffer_model, fps=1, duration=1, loop=0)

#export_visualisation_3d()